In [1]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Image
im = 'https://ultralytics.com/images/zidane.jpg'

# Inference
results = model(im)

results.pandas().xyxy[0]
#      xmin    ymin    xmax   ymax  confidence  class    name
# 0  749.50   43.50  1148.0  704.5    0.874023      0  person
# 1  433.50  433.50   517.5  714.5    0.687988     27     tie
# 2  114.75  195.75  1095.0  708.0    0.624512      0  person
# 3  986.00  304.00  1028.0  420.0    0.286865     27     tie


/home/alexander/.local/lib/python3.10/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/alexander/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-22 Python-3.10.11 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti with Max-Q Design, 4041MiB)



requirements: /home/alexander/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 33.8MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


,xmin,ymin,xmax,ymax,confidence,class,name
0,742.974854,48.395508,1141.844482,720.000000,0.881052,0,person
1,442.007629,437.522400,496.653992,709.973572,0.675213,27,tie
2,123.024139,193.287384,715.662231,719.723877,0.665815,0,person
3,982.803162,308.417358,1027.365845,419.987000,0.260075,27,tie


In [2]:
import cv2
import torch
from PIL import Image

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Images
for f in 'zidane.jpg', 'bus.jpg':
    torch.hub.download_url_to_file('https://ultralytics.com/images/' + f, f)  # download 2 images
im1 = Image.open('zidane.jpg')  # PIL image
im2 = cv2.imread('bus.jpg')[..., ::-1]  # OpenCV image (BGR to RGB)

# Inference
results = model([im1, im2], size=640) # batch of images

# Results
results.print()  
results.save()  # or .show()

results.xyxy[0]  # im1 predictions (tensor)
results.pandas().xyxy[0]  # im1 predictions (pandas)
#      xmin    ymin    xmax   ymax  confidence  class    name
# 0  749.50   43.50  1148.0  704.5    0.874023      0  person
# 1  433.50  433.50   517.5  714.5    0.687988     27     tie
# 2  114.75  195.75  1095.0  708.0    0.624512      0  person
# 3  986.00  304.00  1028.0  420.0    0.286865     27     tie

Using cache found in /home/alexander/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-22 Python-3.10.11 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti with Max-Q Design, 4041MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


requirements: /home/alexander/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 165k/165k [00:00<00:00, 8.08MB/s]
100%|██████████| 476k/476k [00:00<00:00, 12.1MB/s]
image 1/2: 720x1280 2 persons, 2 ties
image 2/2: 1080x810 4 persons, 1 bus
Speed: 13.7ms pre-process, 23.2ms inference, 1.1ms NMS per image at shape (2, 3, 640, 640)
Saved 2 images to runs/detect/exp


,xmin,ymin,xmax,ymax,confidence,class,name
0,742.550171,48.037018,1141.204712,716.641724,0.881825,0,person
1,442.059509,437.528076,496.809326,709.838989,0.687342,27,tie
2,125.191406,193.680664,711.992615,713.046997,0.639421,0,person
3,982.893127,308.356750,1027.368774,420.091736,0.262013,27,tie


## Actividad 14
Investigue sobre la arquitectura de los modelos YOLO, y explique, en no más de tres lı́neas cada uno, los siguientes conceptos:

**Diferencia entre frameworks de detección de objetos de una y dos etapas (single-stage vs. two-stage object detection).**

Los modelos two-stage object detection tienen dos etapas:
* Propuesta de región, realizado por lo que se denomina Region Proposal Network (RPN), la cual "decide" donde mirar para reducir los requerimientos computacionales de todo el proceso de inferencia. La RPN escanea toda locación para evaluar si se requiere procesar una determinada región. Lo hace proponiedo k boxes cada una con dos puntajes (scores) representando la probabilidad o no de un objeto. 
* Clasificación del contenido de los boxes y ajuste de sus coordenadas.
Ejemplo: Faster RCNN. 

Los modelos single-stage detection no tienen una etapa de propuesta de región y entregan una localización final y predicción de contenido a la vez. Esto hace que estos modelos sean más rápidos pero menos precisos. Ejemplo: YOLOv5.

**Función de pérdida Complete Intersection Over Union (BCE) y su uso en la red YOLOv5.**

IoU es una métrica usada para evaluar la performance de un algoritmo en detectar objetos dentro de una imagen. Es calculada por el ratio de la intersección (overlap) de la bounding box predecida y su ground truth, sobre la unión de ambos boxes. Donde 1 indica un overlap perfecto y 0 que no hay overlap. $IoU(A,B)=\frac{A \cap B}{A \cup B}$. Yolo utiliza esta métrica para el cálculo de la precisión promedio utilizando un umbral de precisión en un rango de 0.5 a 0.95.

**Función de pérdida Binary Cross Entropy (BCE) y su uso en la red YOLOv5.**



**¿Cómo se calcula la función de pérdida total en YOLOv5?**

## Actividad 15
Investigue sobre el output de la red YOLOv5 y cómo se traduce el tensor de salida a bounding boxes y detec-
ciones de objetos. Además, explique cómo –en general– se obtiene solamente una detección por objeto, y no
varias para todas las regiones donde el objeto está presente.

## Actividad 16
Instancie el modelo y obtenga la salida para tres fotos a elegir por usted (existen fotos pre-cargadas en Py-
Torch si quieren simplicidad). Para esto, complete y utilice las siguientes funciones:

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
def run_inference(model, img, thresh):
"""
Dado un modelo, una imagen y un umbral, retorna todos los Bounding Boxes y
clases detectada en la imagen que estan por sobre el umbral.
"""
# Deben cambiar la transformada para que funcione
# transform = Compose[...]
# img = transform(img)
# Obtener predicciones
model = model.eval()
predictions = model(img)
# Desempaquetar predicciones y filtrar usando el umbral
detections = {}
# ...
return detections
def visualize_detections(detections):
"""
Dado un diccionario de detecciones, plotea la imagen y cada Bounding Box,
junto con la clase y Objectness Score de cada deteccion
"""
pass